In [3]:
import socket
import hashlib
import random
import string
import paxos_pb2
import threading
import pickle
import socket
import time
import sys
from os import path
from struct import pack, unpack

In [17]:
class State:
    def __init__(self):
        self.balance = 100
        self.blockChain = []
        self.queue = []
        self.clock = 0

In [36]:
# tested = State()
# tested.balance = 1000
# newla = paxos_pb2.Accept()
# newla.type = 2
# tested.blockChain.append(newla)

In [37]:
# with open("stes.txt", "wb") as f:
#     pickle.dump(tested, f)

In [38]:
# tested.balance = 100

In [39]:
# with open("stes.txt", "rb") as f:
#     tested = pickle.load(f)

In [41]:
# tested.balance
# tested.blockChain[0].SerializePartialToString()

b'\x08\x02'

In [ ]:
states = State()

In [2]:
# print("This is stub")
procNo = 0
# tes = hashlib.sha256("nihao".encode("utf-8")).hexdigest()
# print(findNonce(tes))
ports = {1: 10001, 2:10002, 3:10003, 4:10004, 5:10005}
socks = {}
sockLocks = {}
promiseCount = 1
PCLock = threading.Lock()
countLock = threading.Lock()
failed = False
enough = threading.Condition()

for i in range(5):
    socks[i + 1] = -1
    sockLocks[i + 1] = threading.Lock()
    
linkSuc = [False] * 5
activeFailed = [False] * 5

In [28]:
if path.isfile("State_" + str(procNo)):
    with open("State_" + str(procNo)) as f:
        states = pickle.load(f)

In [ ]:
def acceptAndSet(sock):
    

In [ ]:
def failLink():
    global linkSuc, procNo
    print("Please enter the pid of the link to be failed")
    x = getInt()
    if x not in [1,2,3,4,5] or x == procNo:
        print("Invalid pid")
        return
    
    linkSuc[x - 1] = False

In [ ]:
def fixLink():
    global linkSuc
    print("Please enter the pid of the link to be fixed")
    x = getInt()
    if x not in [1,2,3,4,5]:
        print("Invalid pid")
        return
    
    linkSuc[x-1] = True

In [11]:
def getInt():
    while True:
        x = input()
        try:
            x = int(x)
        except ValueError:
            print("Not an integer")
            continue
        
        return x

In [8]:
def helpSend(sock, ite):
    it = ite.SerializeToString()
    le = len(it)
    le = pack(">H", le)
    sock.send(le + it)

In [ ]:
def failProcess():
    global procNo, states
    with open("State_" + str(procNo), "wb") as f:
        pickle.dump(states, f)
        
    print("Failing")
    sys.exit()

In [ ]:
def printChain():
    global states
    print(states.blockChain)

In [ ]:
def printBalance():
    global states
    print(states.balance)

In [ ]:
def printQueue():
    global states
    print(states.queue)

In [ ]:
def comOther()

In [ ]:
def conAndListen(sock, port):
    sock.connect(("localhost", port))
    server_address = ("localhost", ports[procNo])

In [6]:
def randStr():
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(16))

SyntaxError: invalid syntax (<ipython-input-6-15029032fdb1>, line 2)

In [3]:
def findNonce(inp):
    while True:
        st = randStr()
        te = inp + st
        te = te.encode("utf-8")
        ans = hashlib.sha256(te).hexdigest()
        if ans[-1] in ['0', '1', '2', '3', '4']:
            return st

In [ ]:
def incrementCount():
    countLock.acquire()
    count += 1
    countLock.release()

In [ ]:
def timedRec(sock, n, ti):
    mes = []
    count = n
    now = time.time()
    
    while count > 0:
        a = sock.recv(count)
        mes.append(mes)
        le = len(a)
        count -= le
        
        if time.time() - now > ti:
            return False
        
    return b''.join(mes)

In [ ]:
def safeRec(sock, n):
    mes = []
    count = n
    while count > 0:
        a = sock.recv(count)
        mes.append(mes)
        le = len(a)
        count -= le
        
    return b''.join(mes)

In [ ]:
def sendPrepare(sock, sno):
    global ports, procNo, states
    pMes = paxos_pb2.Prepare()
    pMes.type = 1
    pMes.ballot.depth = len(states.blockChain)
    pMes.ballot.pid = procNo
    
    time.sleep(5)
    
    helpSend(sock, pMes)

In [ ]:
def recvPromise(sock):
    global promiseCount
    le = timedRec(+)

In [ ]:
def askForPromise(i):
    global ports, linkSuc, socks
    if linkSuc[i] == False:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_address = ("localhost", ports[i])
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        sock.bind(server_address)
        try:
            sock.connect(server_address)
        except socket.error, msg:
            print("Connection failed " + msg)
            return
        
        socks[i] = sock
        linkSuc[i-1] = True
        
    
    threading.Thread(target = sendPrepare, args = (socks[i], i, ))
    threading.ThreadrecvPromise(socks[i])

In [ ]:
def ElectAsLeader():
    global procNo, enough, count, activeFailed
    for i in [1,2,3,4,5]:
        if i != procNo and activeFailed[i] == False:
            threading.Thread(target = askForPromise, args = (i,))
        
    enough.acquire()
    enough.wait()
    enough.release()
    
    if count < 3:
        count = 1
        print("Getting promise failed, not enough servers")
        return False
    
    count = 1
    return True
    
        

In [ ]:
def transPrepare():
    global procNo
    

In [ ]:
def moneyTransfer():
    global linkSuc, procNo
    print("Please specify the destination pid:")
    x = getInt()
    if x not in [1,2,3,4,5] or x == procNo:
        print("Not a valid pid")
        return False
    
    print("Please specify the amount to transfer:")
    y = getInt()
    if y < 0:
        print("Amount cannot be negative")
        return False
    elif y > balance:
        print("Balance not enough")
        return False
    
    res = ElectAsLeader()
    if res == True:
        res = transPrepare()
        if res == True:
            sendDecide()
        else:
            print("Accepting phase failed")
            return False
    else:
        print("Failed to become leader")
        return False
    
    return True
    

In [4]:
dir(paxos_pb2)

['Accept',
 'Accepted',
 'BallotNum',
 'Block',
 'DESCRIPTOR',
 'Decide',
 'Prepare',
 'Promise',
 'Transaction',
 '_ACCEPT',
 '_ACCEPTED',
 '_BALLOTNUM',
 '_BLOCK',
 '_DECIDE',
 '_PREPARE',
 '_PROMISE',
 '_TRANSACTION',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_b',
 '_descriptor',
 '_message',
 '_reflection',
 '_sym_db',
 '_symbol_database',
 'sys']

In [10]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ("localhost", ports[procNo])
sock.bind(server_address)
sock.listen()

threading.Thread(target=acceptAndSet, args = (sock,))

while True:
    print("Please choose your options:")
    print("1. Money Transfer")
    print("2. Fail link")
    print("3. Fix link")
    print("4. Fail this process")
    print("5. Print block chain")
    print("6. Print balance")
    print("7. Print queue")
    x = getInt()
    if x not in [1,2,3,4,5,6,7]:
        print("Not an valid option!")
        continue
        
    if x == 1:
        moneyTransfer()
    elif x == 2:
        failLink()
    elif x == 3:
        fixLink()
    elif x == 4:
        failProcess()
    elif x == 5:
        printChain()
    elif x == 6:
        printBalance()
    elif x == 7:
        printQueue()


KeyError: 0